In [42]:
!pip install datasets
!pip install bpemb

In [43]:
import nltk
from bpemb import BPEmb
import string
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from torch import nn
import torch

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

<a href="https://colab.research.google.com/github/Axel0087/NLP2023/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
dataset = load_dataset("copenlu/answerable_tydiqa")

In [45]:
train_set = dataset["train"]
validation_set = dataset["validation"]

In [46]:
def get_answer_start(row):
  return row["annotations"]["answer_start"][0]

def get_answer(row):
  return row["annotations"]["answer_text"][0]

def get_document(row):
  return row["document_plaintext"]

def get_question(row):
  return row["question_text"]

def oracle(answer, document):
  return answer != "" and answer in document

def get_language(dataset, lang):
  return [row for row in dataset if row['language'] == lang]

In [47]:
train_arabic = get_language(train_set, "arabic")
val_arabic = get_language(validation_set, "arabic")

train_bengali = get_language(train_set, "bengali")
val_bengali = get_language(validation_set, "bengali")

train_indonesian = get_language(train_set, "indonesian")
val_indonesian = get_language(validation_set, "indonesian")

In [48]:
def ratio_string(train, val):
  val_ratio = round(len(val)/len(train)*100)
  train_ratio = 100-val_ratio
  return f"{train_ratio} / {val_ratio}"

def answerable_ratio(ds):
  answerable = round(sum([1 for row in ds if get_answer_start(row) == -1])/len(ds)*100)
  nonansw = 100-answerable
  return f"{answerable} / {nonansw}"

print(f"""
Dataset features:

{train_set.column_names}

Dataset sizes:

(Arabic) Training set:                                          {len(train_arabic)}
(Arabic) Validation set:                                        {len(val_arabic)}
(Arabic) Ratio (Training/Val):                                  {ratio_string(train_arabic, val_arabic)}
(Arabic) Training balance (Answerable / Not answerable):        {answerable_ratio(train_arabic)}
(Arabic) Validation balance (Answerable / Not answerable):      {answerable_ratio(val_arabic)}

(Bengali) Training set:                                         {len(train_bengali)}
(Bengali) Validation set:                                       {len(val_bengali)}
(Bengali) Ratio (Training/Val):                                 {ratio_string(train_bengali, val_bengali)}
(Bengali) Training balance (Answerable / Not answerable):       {answerable_ratio(train_bengali)}
(Bengali) Validation balance (Answerable / Not answerable):     {answerable_ratio(val_bengali)}

(Indonesian) Training set:                                      {len(train_indonesian)}
(Indonesian) Validation set:                                    {len(val_indonesian)}
(Indonesian) Ratio (Training/Val):                              {ratio_string(train_indonesian, val_indonesian)}
(Indonesian) Training balance (Answerable / Not answerable):    {answerable_ratio(train_indonesian)}
(Indonesian) Validation balance (Answerable / Not answerable):  {answerable_ratio(val_indonesian)}
""")


Dataset features:

['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url']

Dataset sizes:

(Arabic) Training set:                                          29598
(Arabic) Validation set:                                        1902
(Arabic) Ratio (Training/Val):                                  94 / 6
(Arabic) Training balance (Answerable / Not answerable):        50 / 50
(Arabic) Validation balance (Answerable / Not answerable):      50 / 50

(Bengali) Training set:                                         4779
(Bengali) Validation set:                                       224
(Bengali) Ratio (Training/Val):                                 95 / 5
(Bengali) Training balance (Answerable / Not answerable):       50 / 50
(Bengali) Validation balance (Answerable / Not answerable):     50 / 50

(Indonesian) Training set:                                      11394
(Indonesian) Validation set:                                    1191
(Indonesian) Ra

In [8]:
def bag_of_words(dataset, column):
  bag = {}
  for row in dataset:
    tokens = nltk.word_tokenize(row[column])

    for token in tokens:

      if not token in bag:
        bag[token] = 0

      bag[token] += 1
      #print(bag)
  return sorted(bag.items(), key=lambda item: item[1], reverse=True)

#def sort_bag(bag):
#  return sorted(bag.items(), key=lambda item: item[1], reverse=True)

In [9]:
arabic_doc_bow = bag_of_words(train_arabic, "document_plaintext")
arabic_question_bow = bag_of_words(train_arabic, "question_text")

bengali_doc_bow = bag_of_words(train_bengali, "document_plaintext")
bengali_question_bow = bag_of_words(train_bengali, "question_text")

indonesian_doc_bow = bag_of_words(train_indonesian, "document_plaintext")
indonesian_question_bow = bag_of_words(train_indonesian, "question_text")

In [10]:
print(f"""

Most common words:

(Arabic) Documents: {arabic_doc_bow[0:5]}
(Arabic) Questions: {arabic_question_bow[0:5]}

(Bengali) Documents: {bengali_doc_bow[0:5]}
(Bengali) Questions: {bengali_question_bow[0:5]}

(Indonesian) Documents: {indonesian_doc_bow[0:5]}
(Indonesian) Questions: {indonesian_question_bow[0:5]}
""")



Most common words:

(Arabic) Documents: [('في', 89705), ('.', 88299), ('من', 61719), ('[', 38120), (']', 38119)]
(Arabic) Questions: [('؟', 10061), ('ما', 7451), ('متى', 7130), ('هو', 6760), ('من', 6309)]

(Bengali) Documents: [(',', 12184), (']', 7123), ('[', 7120), ('ও', 5195), ('এবং', 5102)]
(Bengali) Questions: [('?', 4777), ('কী', 940), ('নাম', 837), ('কত', 802), ('হয়', 800)]

(Indonesian) Documents: [(',', 54165), ('.', 43063), ('yang', 24077), ('dan', 23741), ('di', 16604)]
(Indonesian) Questions: [('?', 11368), ('yang', 1814), ('Kapan', 1811), ('Apa', 1633), ('Apakah', 1227)]



In [11]:
def get_ratio(question, document, stop_words):
  tokens = nltk.word_tokenize(question)
  count = 0
  stripped_tokens = set(tokens) - stop_words
  for token in stripped_tokens:
    if token in document:
      count += 1
  return count/len(stripped_tokens)


def avg(lst):
  return sum(lst)/len(lst)

def get_average_ratios(training, stop_words):
  answerable_ratios = []
  nonanswerable_ratios = []
  for row in training:
    ratio = get_ratio(get_question(row), get_document(row), stop_words)
    lst = answerable_ratios if oracle(get_answer(row), get_document(row)) else nonanswerable_ratios
    lst.append(ratio)
  return avg(answerable_ratios), avg(nonanswerable_ratios)

class NaiveModel:
  def __init__(self, stop_words):
    self.stop_words = stop_words
    self.ratio = -1

  def train(self, training):
    answerable_ratio, nonanswerable_ratio = get_average_ratios(training, self.stop_words)
    self.ratio = (answerable_ratio + nonanswerable_ratio)/2

  def classify(self, question, document):
    return get_ratio(question, document, self.stop_words) > self.ratio

def evaluate(validation, model):
  res = [int(oracle(get_answer(row), get_document(row)) == model.classify(get_question(row), get_document(row))) for row in validation]
  acc = avg(res)

  ### Manual generation of confusion matrix for scores like Balanced Accuray and F-score
  #tp, fp, tn, fn = 0, 0, 0, 0
  #for row in validation:
  #  gt = oracle(get_answer(row), get_document(row))
  #  cl = model.classify(get_question(row), get_document(row))
  #  if (cl):
  #    if (gt):
  #      tp += 1
  #    else:
  #      fp += 1
  #  else:
  #    if (gt):
  #      fn += 1
  #    else:
  #      tn += 1
  #tpr = tp / (tp + fn)
  #tnr = tn / (tn + fp)
  #ba = (tpr + tnr) / 2

  print(f"Accuracy: {round(acc*100, 4)}%\n")

In [12]:
from nltk.corpus import stopwords

nltk.download('stopwords')

arabic_stop_words = set(stopwords.words('indonesian')) | set(string.punctuation) | set("؟")
bengali_stop_words = set(stopwords.words('bengali')) | set(string.punctuation)
indonesian_stop_words = set(stopwords.words('indonesian')) | set(string.punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
print("Evaluating arabic:")

arabic_model = NaiveModel(arabic_stop_words)
arabic_model.train(train_arabic)
evaluate(val_arabic, arabic_model)

print("Evaluating bengali:")

bengali_model = NaiveModel(bengali_stop_words)
bengali_model.train(train_bengali)
evaluate(val_bengali, bengali_model)

print("Evaluating indonesian:")

indonesian_model = NaiveModel(indonesian_stop_words)
indonesian_model.train(train_indonesian)
evaluate(val_indonesian, indonesian_model)

Evaluating arabic:
Accuracy: 71.6614%

Evaluating bengali:
Accuracy: 72.3214%

Evaluating indonesian:
Accuracy: 71.2007%



In [49]:
vocab_size = 25000
encoding_dim = 100

bpemb_ar = BPEmb(lang='ar', dim=encoding_dim, vs=vocab_size)
bpemb_bn = BPEmb(lang='bn', dim=encoding_dim, vs=vocab_size)
bpemb_in = BPEmb(lang='id', dim=encoding_dim, vs=vocab_size)


In [15]:
train_arabic_doc = [get_document(row) for row in train_arabic]
train_arabic_question = [get_question(row) for row in train_arabic]

train_bengali_doc = [get_document(row) for row in train_bengali]
train_bengali_question = [get_question(row) for row in train_bengali]

train_indonesian_doc = [get_document(row) for row in train_indonesian]
train_indonesian_question = [get_question(row) for row in train_indonesian]

In [126]:
def get_bpemb_features(dataset, bpemb):
  return [bpemb.embed(x) for x in tqdm(dataset)]

def text_to_ids(text, tokenizer):
    input_ids = tokenizer.encode_ids_with_eos(text)
    return input_ids, len(input_ids)

def pad_input(input):
    input_ids = [i[0] for i in input]
    seq_lens = [i[1] for i in input]

    max_length = max(seq_lens)

    input_ids = [(i + [25000] * (max_length - len(i))) for i in input_ids]

    # Make sure each sample is max_length long
    assert (all(len(i) == max_length for i in input_ids))
    return torch.tensor(input_ids), torch.tensor(seq_lens)

In [51]:
!pip install torcheval

In [60]:
from torch.utils.data import DataLoader, Dataset

class DatasetReader(Dataset):
  def __init__(self, data, tokenizer):
    self.data = data
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    row = self.data[idx]
    # Calls the text_to_batch function
    input_ids, seq_lens = text_to_ids(row, self.tokenizer)
    return input_ids, seq_lens


In [63]:

reader[0]

([913, 20783, 1062, 425, 9712, 1614, 326, 20301, 2], 9)

In [127]:
from torcheval.metrics.text import Perplexity

class LSTMNetwork(nn.Module):
    def __init__(
            self,
            pretrained_embeddings,
            vocab_size: int,
            num_layers,
            hidden_dim: int,
            dropout_rate: float = 0.1
    ):
        super(LSTMNetwork, self).__init__()

        self.vocab_size = vocab_size

        self.embeddings = nn.Embedding.from_pretrained(pretrained_embeddings, padding_idx=pretrained_embeddings.shape[0] - 1)

        self.lstm = nn.LSTM(
                pretrained_embeddings.shape[1],
                hidden_dim,
                num_layers,
                batch_first=True,
                dropout=dropout_rate)

        self.dropout = nn.Dropout(dropout_rate)

        self.output_layer = nn.Linear(hidden_dim, vocab_size)

        # Initialize the weights of the model
        self._init_weights()

    def _init_weights(self):
        all_params = list(self.lstm.named_parameters()) + list(self.output_layer.named_parameters())
        for n,p in all_params:
            if 'weight' in n:
                nn.init.xavier_normal_(p)
            elif 'bias' in n:
                nn.init.zeros_(p)

    def forward(self, inputs, input_lens):

        embeds = self.embeddings(inputs)

        # Pack padded: This is necessary for padded batches input to an RNN
        lstm_in = nn.utils.rnn.pack_padded_sequence(
            embeds,
            input_lens.cpu(),
            batch_first=True,
            enforce_sorted=False
        )

        lstm_out, _ = self.lstm(lstm_in)

        lstm_out, _ = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)

        targets = torch.flatten(inputs.clone())

        output = self.output_layer(lstm_out)
        logits = output.view(-1, self.vocab_size)

        loss_fn = nn.NLLLoss()
        loss = loss_fn(logits, targets)

        return (output, loss)


In [166]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [186]:
def train(
    model,
    dl,
    optimizer: torch.optim.Optimizer,
    n_epochs: int,
    device,
    patience: int = 10
):
  # Keep track of the loss and best accuracy
  best_ppl = float("inf")
  pcounter = 0

  # Iterate through epochs
  for ep in range(n_epochs):

    logits_epoch = torch.tensor([]).to(device)
    inputs_epoch = torch.tensor([]).int().to(device)

    for batch in tqdm(dl):
      model.train()
      optimizer.zero_grad()

      batch = tuple(t.to(device) for t in batch)
      inputs = batch[0]

      seq_lens = batch[1]

      (logits, loss) = model(inputs, seq_lens)

      logits_epoch = torch.cat((logits_epoch, logits), 0)
      inputs_epoch = torch.cat((inputs_epoch, inputs), 0)

      loss.backward()
      optimizer.step()

    targets = inputs_epoch.clone().to(device)

    metric=Perplexity()

    metric.update(logits_epoch.cpu(), targets.cpu())
    ppl = metric.compute()

    print(f'Perplexity at epoch {ep}: {ppl}')

    # Keep track of the best model based on the accuracy
    if ppl < best_ppl:
      torch.save(model.state_dict(), 'best_model')
      best_ppl = ppl
      pcounter = 0
    else:
      pcounter += 1
      if pcounter == patience:
        break

  model.load_state_dict(torch.load('best_model'))
  return model

In [187]:
from torch.optim import Adam

num_layers = 2
hidden_dim = 100
dropout_rate = 0.1
lr = 3e-4
n_epochs = 200
batch_size = 32

device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")

pretrained_embeddings = torch.Tensor(np.concatenate([bpemb_bn.emb.vectors, np.zeros(shape=(1,100))], axis=0))
vocabulary = bpemb_bn.emb.index_to_key + ['[PAD]']

model = LSTMNetwork(pretrained_embeddings, len(vocabulary), num_layers, hidden_dim, dropout_rate).to(device)

reader = DatasetReader(train_bengali_question[0:5], bpemb_bn)

dl = DataLoader(reader, batch_size=batch_size, collate_fn=pad_input, shuffle=True, num_workers=2)

optimizer = Adam(model.parameters(), lr=lr)

train(model, dl, optimizer, n_epochs, device)

100%|██████████| 1/1 [00:00<00:00,  1.48it/s]


Perplexity at epoch 0: 25004.836478282516


100%|██████████| 1/1 [00:00<00:00,  1.78it/s]


Perplexity at epoch 1: 24981.7307099388


100%|██████████| 1/1 [00:00<00:00,  3.62it/s]


...

100%|██████████| 1/1 [00:00<00:00,  5.38it/s]


Perplexity at epoch 198: 399.90865773349054


100%|██████████| 1/1 [00:00<00:00,  5.55it/s]


Perplexity at epoch 199: 398.2570196480786


LSTMNetwork(
  (embeddings): Embedding(25001, 100, padding_idx=25000)
  (lstm): LSTM(100, 100, num_layers=2, batch_first=True, dropout=0.1)
  (dropout): Dropout(p=0.1, inplace=False)
  (output_layer): Linear(in_features=100, out_features=25001, bias=True)
)

In [134]:
for smt in dl:
  print(smt)
  break


(tensor([[ 8074,  4662,    22, 24797,  8988,  6078,    83, 11372, 11747,   376,
         13826, 12717,    78, 20301,     2],
        [ 6726,  4136, 24798,  1121,  7705,   779,   326, 20301,     2, 25000,
         25000, 25000, 25000, 25000, 25000],
        [  913, 20783,  1062,   425,  9712,  1614,   326, 20301,     2, 25000,
         25000, 25000, 25000, 25000, 25000],
        [ 4647,   145, 22525,  8841,  6520,  2049,    78, 20301,     2, 25000,
         25000, 25000, 25000, 25000, 25000],
        [ 4647,   145,  1998,  3416,   139,    78, 20301,     2, 25000, 25000,
         25000, 25000, 25000, 25000, 25000]]), tensor([15,  9,  9,  9,  8]))
